### Jupyter Notebook to evaluate the fine-tuned YOLOv8s model on the VisDrone dataset

In [ ]:
from src.dataset import VisDroneDetectionDataset, collate_fn
from torch.utils.data import DataLoader
from torchvision import transforms
from src.evaluate import ObjectDetectionEvaluator
from src.models import YOLODetectionModel
from src.annotations import VisDroneClasses, LabelsConverter
import os
import torch
import argparse

In [ ]:
annot_dir = './data/VisDrone2019-MOT-test-dev/annotations'
annot_files = [os.path.join(annot_dir, f) for f in os.listdir(annot_dir)]

images_directory = './data/VisDrone2019-MOT-test-dev/sequences'

In [ ]:
resize = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda img: transforms.functional.pad(img, (0, 0, (32 - img.shape[-1] % 32) % 32, (32 - img.shape[-2] % 32) % 32))),
])

dataset = VisDroneDetectionDataset(annot_files, images_directory , transform=resize)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2, collate_fn=collate_fn)

In [ ]:
print(f"Dataset size: {len(dataset)} images")

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
evaluator = ObjectDetectionEvaluator(device=device)

In [ ]:
yolo_vis_converter = LabelsConverter()
for class_id in VisDroneClasses:
    yolo_vis_converter.rules[class_id] = class_id + 1
model = YOLODetectionModel(model_path='./data/models/checkpoints/yolov8s-VisDrone.pt', yolo_classes=yolo_vis_converter).to(device)

In [ ]:
results = evaluator.evaluate_mAP(model, data_loader)

In [ ]:
metric = evaluator.metric


In [ ]:
import matplotlib.pyplot as plt

def plot_per_class_map(results_dict, class_names=None):
    mAPs = results_dict["map_per_class"].detach().cpu().numpy()
    classes = results_dict["classes"].detach().cpu().numpy()

    if class_names:
        labels = [class_names[i] for i in classes]
    else:
        labels = [f"Class {i}" for i in classes]

    plt.figure(figsize=(10, 6))
    plt.bar(labels, mAPs, color='skyblue', edgecolor='navy')
    plt.xlabel('Classes')
    plt.ylabel('mAP')
    plt.title('Mean Average Precision (mAP) per Class')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 1.0) # mAP is usually 0 to 1
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

plot_per_class_map(results, class_names=[c.name for c in VisDroneClasses])